#Librerias 

In [ ]:
import gc
import networkx as nx

##Clase Nodo

In [ ]:
class Node:
    def __init__(self, value=None):
        self.value = value
        self.next = None
        self.prev = None



##Clase Lista doblemente Enlazada

In [ ]:
class DoubleLinkedList:

    def __init__(self):
        self.start = Node()
        self.end = Node()
        self.size = 0

    def __len__(self):
        return self.size

    def __iter__(self):
        node = self.start
        while node:
            yield node.value
            node = node.next

    def push_front(self, value):
        new = Node(value)
        if self.size == 0:
            self.start = self.end = new
            self.size += 1
            return
        new.next = self.start
        self.start.prev = new
        self.start = new
        self.size += 1

    def push_back(self, value):
        if self.size == 0:
            self.push_front(value)
            return
        new = Node(value)
        self.end.next = new
        new.prev = self.end
        self.end = new
        self.size += 1

    def insert(self, index, value):
        if self.size == 0 or index == 0:
            self.push_front(value)
            return
        if index == self.size - 1:
            self.push_back(value)
            return
        if index < 0 or index >= self.size:
            return
        new = Node(value)
        c = self.start
        for i in range(index - 1):
            c = c.next
        new.next = c.next
        c.next.prev = new
        c.next = new
        new.prev = c
        self.size += 1

    def pop_back(self):
        if self.size <= 0:
            return
        if self.size == 1:
            self.start = self.end = None
            self.size -= 1
            gc.collect()  # eliminar memoria sin referencia
            return
        self.end = self.end.prev
        self.end.next.prev = None
        self.end.next = None
        self.size -= 1
        gc.collect()  # eliminar memoria sin referencia

    def pop_front(self):
        if self.size <= 1:
            self.pop_back()
            return
        self.start = self.start.next
        self.start.prev.next = None
        self.start.prev = None
        self.size -= 1
        gc.collect()

    def delete(self, index):
        # lo hacen uds, eliminar en una posicion :v
        self.size -= 1

    def erase(self, pos):
        if pos < self.size and pos >= 0:
            if (pos == 0):
                self.pop_front()
            elif pos == self.size:
                self.pop_back()
            else:
                punt = self.start
                contador = 0
                while (contador != pos - 1):
                    punt = punt.next
                    contador += 1
                    elimi = punt.next
                punt.next = elimi.next
                elimi.next = None
                del elimi
                self.size -= 1

    def buscar_por_valor(self, valor):
        contador = 0
        punt = self.start
        while (contador != self.size):
            if punt.value == valor:
                return contador
            punt = punt.next
            contador += 1
        return -1


##Clase Quoridor

In [ ]:
class Quoridor:
  def init(self, n=5, q_players = 2):

        self.n = n
        self.q_nodos = n*n
        self.q_players = 2
        self.grafo = [DoubleLinkedList() for i in range(self.q_nodos)]
        for i, nodo in enumerate(self.grafo):
          if i % n != n - 1:
            self.grafo[i].push_back(i + 1)
            self.grafo[i + 1].push_back(i)
          if i + n < n*n:
            self.grafo[i].push_back(i+n)
            self.grafo[i + n].push_back(i)
        dict = []
        for i, nodo in enumerate(self.grafo):
          for j in list(nodo):
              data = (i,j)
              dict.append(data)
        return dict

  def dicionario(self):
      dic = []
      listaN = [x for x in range(self.q_nodos)]
      for i, nodo in enumerate(self.grafo):
          data = (list(nodo))
          dic.append(data)
      diccio = (dict(zip(listaN, dic)))
      return diccio

  def dijsktra(self, Nodos):

    grafo = self.dicionario()

    S = [];
    Queue = [];
    anterior = [0 for i in range(max(grafo) +1)];
    distancia = [0 for i in range(max(grafo) +1)]

    for nodo in grafo:
        distancia[nodo] = 10000
        Queue.append(nodo)
    distancia[Nodos[0]] = 0

    while not len(Queue) == 0:
        distancia_minima = 10000
        for nodo in Queue:
            if distancia[nodo] < distancia_minima:
                distancia_minima = distancia[nodo]
                nodo_temporal = nodo
        nodo_distancia_minima = nodo_temporal
        Queue.remove(nodo_distancia_minima)

        for vecino in grafo[nodo_distancia_minima]:
            if distancia[nodo_distancia_minima] == 10000:
                distancia_temporal = 0
            else:
                distancia_temporal = distancia[nodo_distancia_minima]
            distancia_con_peso = distancia_temporal + 1
            if distancia_con_peso < distancia[vecino]:
                distancia[vecino] = distancia_con_peso
                anterior[vecino] = nodo_distancia_minima

        if nodo_distancia_minima == Nodos[1]:
            if anterior[nodo_distancia_minima] != 0 or nodo_distancia_minima == Nodos[0]:
                while nodo_distancia_minima != 0:
                    S.insert(0, nodo_distancia_minima)
                    nodo_distancia_minima = anterior[nodo_distancia_minima]
                return S
                
  def bellman_ford(self, start, end):
    ruta = DoubleLinkedList()
    ruta.push_front(end)
    distancia = [float("Inf") for x in range(self.q_nodos)]
    previo = [-1 for x in range(self.q_nodos)]
    distancia[start] = 0
    for iteracion in range(self.q_nodos - 1):
      for actual, nodo in enumerate(self.grafo):
        for adyacente in nodo:
          if distancia[actual] + 1 < distancia[adyacente]:
            distancia[adyacente] = distancia[actual] + 1
            previo[adyacente] = actual
    anterior = previo[end]
    while anterior != -1:
      ruta.push_front(anterior)
      anterior = previo[anterior]
    return list(ruta)

  def h(self, start, end):
    ys =  start.value % self.n
    xs = start.value // self.n + ys
    ye = end.value % self.n
    xe = end.value // self.n + ye
    return abs(xs - xe) + abs(ys - ye)              

               

#Algoritmo A Star

In [ ]:
def heuristic(current, goal):
    h = abs(current[0] - goal[0]) + abs(current[1] - goal[1])
    return h


def get_heuristic_value(node):
    return node['global']


def Astar(Graph, start, goal, n):
    current = start
    c = start % n + start // n
    f = start // n
    c1 = goal % n + goal // n
    f1 = goal // n
    G.nodes[start]['local'] = 0
    G.nodes[start]['global'] = heuristic((c, f), (c1, f1))
    not_tested_nodes = [G.nodes[start]]

    while len(not_tested_nodes) != 0 and current != goal:

        not_tested_nodes.sort(key=get_heuristic_value)

        while len(not_tested_nodes) != 0 and not_tested_nodes[0]['visited']:
            not_tested_nodes.pop(0)

        if len(not_tested_nodes) == 0:
            break

        current = not_tested_nodes[0]['pos']
        not_tested_nodes[0]['visited'] = True

        for neighbor in Graph.neighbors(current):
            if not G.nodes[neighbor]['visited']:
                not_tested_nodes.append(G.nodes[neighbor])
            sum = G.nodes[current]['local'] + 1

            if sum < G.nodes[neighbor]['local']:
                G.nodes[neighbor]['padre'] = current
                print(G.nodes[neighbor]['padre'])
                G.nodes[neighbor]['local'] = sum
                c = neighbor % n + neighbor // n
                f = neighbor // n
                G.nodes[neighbor]['global'] = G.nodes[neighbor]['local'] + heuristic((c, f), (c1, f1))

##Algoritmo dijkstra (Explicacion algoritmo)
def dijsktra(self, Nodos):                 -> funcion dijsktra

    grafo = self.dicionario()              -> diccionario de lista de adyacencia 

    S = [];                                -> lista  
    Queue = [];                            -> cola 
    anterior = [0 for i in range(max(grafo) +1)]; -> lista de 0's con tamaño de grafo para inicializar anterior
    distancia = [0 for i in range(max(grafo) +1)] -> lista de 0's con tamaño con tamaño de grafo para inicializar distancia

    for nodo in grafo:                        ->recorre el grafo lo añade a una cola retornando la cantidad de nodos en una lista 
        distancia[nodo] = 10000
        Queue.append(nodo)
    distancia[Nodos[0]] = 0    -> distancia recorrida en primer nodo es 0

    while not len(Queue) == 0:        -> mientras el tamaño de la lista no sea igual a 0 
        distancia_minima = 10000      -> la distancia minima es igual a 10000
        for nodo in Queue:            ->  recorre la cola/lista 
            if distancia[nodo] < distancia_minima:  -> si la distancia en el nodo es menor que la distancia minima 
                distancia_minima = distancia[nodo]  ->la distancia minima es igual a la distancia en el nodo 
                nodo_temporal = nodo -> crea un nodo temporal
        nodo_distancia_minima = nodo_temporal   -> nodo de la distancia minima lo iguala al nodo temporal
        Queue.remove(nodo_distancia_minima) -> elimina de la cola el nodo

        for vecino in grafo[nodo_distancia_minima]:     -> recorre el nodo vecino en el grafo con la distancia minima
            if distancia[nodo_distancia_minima] == 10000: -> si la distancia del nodo de distancia minima es == 100000 
                distancia_temporal = 0  -> la distancia temporal es igual a 0
            else:                       -> por el contrario 
                distancia_temporal = distancia[nodo_distancia_minima]  ->la distancia temporal se iguala a la distancia del nodo de distancia minima
            distancia_con_peso = distancia_temporal + 1      -> creo variable de distancia con peso y la igual a la distancia temporal +1 ( ya que los enlaces valen peso 1)
            if distancia_con_peso < distancia[vecino]:  -> si la distancia con peso es menor a la distancia del vecino 
                distancia[vecino] = distancia_con_peso  -> la distancia del vecino es igual a la distancia con peso
                anterior[vecino] = nodo_distancia_minima -> la ditancia del vecino anterior es igual a la distancia minima

        if nodo_distancia_minima == Nodos[1]:        -> si el nodo de distancia minima es igual a Nodos[1]
            if anterior[nodo_distancia_minima] != 0 or nodo_distancia_minima ==  Nodos[0]: -> si si la distancia anterior es diferente que 0 o el nodo distancia minima es igual a Nodos[0] 
                while nodo_distancia_minima != 0:      -> mientras que la distancia minima sea diferente a 0
                    S.insert(0, nodo_distancia_minima)     -> se inserta en la lista S nodo con la distancia minima
                    nodo_distancia_minima = anterior[nodo_distancia_minima] -> nodo de distancia minima se iguala a nodo de distancia minima anterior
                return S  -> retorna en forma de lista S

##Algoritmo A star (Explicacion algoritmo)


###def heuristic(current, goal):              -> funcion que me da la heuristica
    h = abs(current[0] - goal[0]) + abs(current[1] - goal[1]) ## Operacion de la heuristica manhattan
    return h

### def get_heuristic_value(node):  -> se usara como lambda 
    return node['global']

###def Astar(Graph, start, goal, n):              -> funcion A star
    current = start  # defino al nodo del inicio como el concurrente
    formula para hallar el punto x de un vertice de la posicion de un nodo del tablero
    c = start % n + start // n -> vertice x de la celda concurrente
    f = start // n -> vertice y de la celda concurrente
    c1 = goal % n + goal // n -> vertice x de la celda a la que queremos llegar
    f1 = goal // n -> vertice y de la celda a la que queremos llegar
    G.nodes[start]['local'] = 0 -> Distancia del nodo en 0 (es el inicio)
    G.nodes[start]['global'] = heuristic((c, f), (c1, f1)) -> Distancia del nodo inicial del nodo final
    not_tested_nodes = [G.nodes[start]] -> creo una lista y agrego al nodo inicial

    while len(not_tested_nodes) != 0 and current != goal: -> mientras que la lista no este vacia y no haya llegado al nodo objetivo

        not_tested_nodes.sort(key=get_heuristic_value) -> ordeno la lista, como si fuera una cola de prioridad

        while len(not_tested_nodes) != 0 and not_tested_nodes[0]['visited']: -> mientras que la lista no este vacia y el nodo del principio haya sido visitado
            not_tested_nodes.pop(0) -> lo elimino

        if len(not_tested_nodes) == 0: -> si veo que ya no hay nodos a comprobar, termino el algoritmo
            break

        current = not_tested_nodes[0]['pos'] -> obtengo la posicion del nodo que está al principio de la lista (constantemente se ordena)
        not_tested_nodes[0]['visited'] = True -> marco como visitado al nodo de la primera posicion de la lista

        for neighbor in Graph.neighbors(current): -> visito a los vecinos del nodo concurrente
            if not G.nodes[neighbor]['visited']: -> si no los he visitado
                not_tested_nodes.append(G.nodes[neighbor]) -> los agrego a mi lista de no comprobados
            sum = G.nodes[current]['local'] + 1 -> mi peso siempre será 1 (+1), le sumo esa cantidad al peso que ya tengo

            if sum < G.nodes[neighbor]['local']: -> si la suma es menor al valor que está actualmente como local en el nodo
                G.nodes[neighbor]['padre'] = current -> el nodo concurrente será el padre del nodo evaluado en esta iteracion
                print(G.nodes[neighbor]['padre'])
                G.nodes[neighbor]['local'] = sum -> su nuevo local, o distancia acumulada será la suma hecha previamente
                c = neighbor % n + neighbor // n -> actualizo los valores para obtener la heuristica de ese nodo
                f = neighbor // n 
                G.nodes[neighbor]['global'] = G.nodes[neighbor]['local'] + heuristic((c, f), (c1, f1)) -> el valor global ahora es el valor de la heuristica más lo que había en el local (actualizado previamente)


##Algoritmo bellman ford (Explicacion algoritmo)
  def bellman_ford(self, start, end):

    ruta = DoubleLinkedList()                       -> variable ruta se inicializa con la lista doblemente enlazada
    ruta.push_front(end)        -> se añade al  comienzo de la lista el nodo final
    distancia = [float("Inf") for x in range(self.q_nodos)]  -> se inicializa la distancia con una lista de infinito con longitud del tablero 
    previo = [-1 for x in range(self.q_nodos)]      -> se inicializa el previo con una lista de -1 con longitud del tablero 
    distancia[start] = 0    -> la distancia en el inicio se inicializa a 0
    for iteracion in range(self.q_nodos - 1):   -> recorrido de iteracion a la cantidad de nodos -1
      for actual, nodo in enumerate(self.grafo): -> recorrido en el grafo
        for adyacente in nodo:  -> recorrido de los nodos adyacentes
          if distancia[actual] + 1 < distancia[adyacente]: -> si la distancia actual +1 es menor a la distancia adyacente
            distancia[adyacente] = -> distancia[actual] + 1 la distancia adyacente es igual a la distancia actual +1
            previo[adyacente] =  -> actual  se actualiza el nodo previo con el actual
    anterior = previo[end]  -> se actualiza el nodo previo final
    while anterior != -1:    -> mientras el nodo anterior no sea igual a -1
      ruta.push_front(anterior)  -> se grega a la lista 
      anterior = previo[anterior] -> se actualiza el anterior al previo del anterior
    return list(ruta) -> returna la lista de la ruta mas corta


#Casos de uso

In [ ]:
juego = Quoridor()
juego.init(5)

print("Dijkstra")
print("Pruebas de caso")
print("5*5----------")
print(f"ruta de 0 a 24 --> {juego.dijsktra((0, 24))}")
print(f"ruta de 22 a 5 --> {juego.dijsktra((22, 5))}")
print(f"ruta de 23 a 9 --> {juego.dijsktra((23, 9))}")

print("4*4----------")
juego.init(4)

print(f"ruta de 0 a 12 --> {juego.dijsktra((0, 12))}")
print(f"ruta de 1 a 9 --> {juego.dijsktra((1, 9))}")
print(f"ruta de 3 a 15 --> {juego.dijsktra((3, 15))}")

print("9*9----------")
juego.init(9)

print(f"ruta de 6 a 76 --> {juego.dijsktra((6, 76))}")
print(f"ruta de 3 a 15 --> {juego.dijsktra((3, 15))}")
print(f"ruta de 6 a 64 --> {juego.dijsktra((6, 64))}")




Dijkstra
Pruebas de caso
5*5----------
ruta de 0 a 24 --> [1, 2, 3, 4, 9, 14, 19, 24]
ruta de 22 a 5 --> [22, 17, 12, 7, 6, 5]
ruta de 23 a 9 --> [23, 18, 13, 8, 9]
4*4----------
ruta de 0 a 12 --> [4, 8, 12]
ruta de 1 a 9 --> [1, 5, 9]
ruta de 3 a 15 --> [3, 7, 11, 15]
9*9----------
ruta de 6 a 76 --> [6, 5, 4, 13, 22, 31, 40, 49, 58, 67, 76]
ruta de 3 a 15 --> [3, 4, 5, 6, 15]
ruta de 6 a 64 --> [6, 5, 4, 3, 2, 1, 10, 19, 28, 37, 46, 55, 64]


In [ ]:
juego.init(3)
print("Bellman Ford")
print("Pruebas de caso")
print("3*3----------")
print(f"ruta de 7 a 1 --> {juego.bellman_ford(1, 7)}")
print(f"ruta de 8 a 3 --> {juego.bellman_ford(8, 3)}")
print(f"ruta de 0 a 8 --> {juego.bellman_ford(0, 8)}")




Bellman Ford
Pruebas de caso
3*3----------
ruta de 7 a 1 --> [1, 4, 7]
ruta de 8 a 3 --> [8, 5, 4, 3]
ruta de 0 a 8 --> [0, 1, 2, 5, 8]


In [ ]:
    juego = Quoridor()
    n = 3
    edges = juego.init(n)
    G = nx.Graph()
    G.add_edges_from(edges)
    i = 0
    for _, u in G.nodes(data=True):
        u['padre'] = None
        u['local'] = float('inf')
        u['global'] = float('inf')
        u['visited'] = False
        u['pos'] = i
        i = i + 1
    print("Algoritmo A star")
    print("Pruebas de caso")
    print("3*3----------")
    print("ruta de 1 a 7 -->")
    Astar(G, 1, 7, n)


Algoritmo A star
Pruebas de caso
3*3----------
ruta de 1 a 7 -->
1
1
1
4
4
4
7
7
